# Segmenting and Clustering neighborhoods in Toronto - Final Assignement
___

## Install & Import Python Modules

In [2]:
!pip install lxml
!pip install geopy
!pip install folium

import numpy as np

import pandas as pd
from lxml import etree

from geopy.geocoders import Nominatim
import folium
import requests
from sklearn.cluster import KMeans 

import matplotlib.cm as cm
import matplotlib.colors as colors

## Read data from Wikipedia and assign to a DataFrame

In [3]:
wikiURL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

WikiData = pd.read_html(wikiURL)

TorontoData = WikiData[0]

# Change Col Names
TorontoData = TorontoData.rename(columns={'Postal Code': 'PostalCode'})

# Check DataFrame
TorontoData

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## Data Cleaning

* ### Remove rows with Borough not assigned

In [4]:
TorontoData.drop(TorontoData.loc[TorontoData['Borough']=='Not assigned'].index, axis=0, inplace=True)

* ### Combine the Neighbourhood with same PostalCode

In [5]:
uPostalCode = TorontoData['PostalCode'].unique()
for i, P in enumerate(uPostalCode):
    SamePostalCode = TorontoData.loc[TorontoData['PostalCode']==P]
    B = TorontoData['Borough'].loc[TorontoData['PostalCode']==P].values[0]
    print('{} Postal Code: {} "{}" ({} Neighbourhood occurrence))'.format(i+1, P, B, SamePostalCode.shape[0]))
    if SamePostalCode.shape[0]>1:
        MultiN = ','.join(SamePostalCode['Neighbourhood'].to_list())
        TorontoData.loc[TorontoData.shape[0]+1] = [P, B, MultiN]
        TorontoData.drop(SamePostalCode.index, axis=0, inplace=True)

1 Postal Code: M3A "North York" (1 Neighbourhood occurrence))
2 Postal Code: M4A "North York" (1 Neighbourhood occurrence))
3 Postal Code: M5A "Downtown Toronto" (1 Neighbourhood occurrence))
4 Postal Code: M6A "North York" (1 Neighbourhood occurrence))
5 Postal Code: M7A "Downtown Toronto" (1 Neighbourhood occurrence))
6 Postal Code: M9A "Etobicoke" (1 Neighbourhood occurrence))
7 Postal Code: M1B "Scarborough" (1 Neighbourhood occurrence))
8 Postal Code: M3B "North York" (1 Neighbourhood occurrence))
9 Postal Code: M4B "East York" (1 Neighbourhood occurrence))
10 Postal Code: M5B "Downtown Toronto" (1 Neighbourhood occurrence))
11 Postal Code: M6B "North York" (1 Neighbourhood occurrence))
12 Postal Code: M9B "Etobicoke" (1 Neighbourhood occurrence))
13 Postal Code: M1C "Scarborough" (1 Neighbourhood occurrence))
14 Postal Code: M3C "North York" (1 Neighbourhood occurrence))
15 Postal Code: M4C "East York" (1 Neighbourhood occurrence))
16 Postal Code: M5C "Downtown Toronto" (1 Neighb

* ### Check for Neighbourhood if equal to 'Not assigned' and assign same value as Borough to Neighbourhood

In [6]:
Idx1 = TorontoData.loc[TorontoData['Neighbourhood']=='Not assigned']
if not Idx1.index.empty:
    TorontoData.loc[Idx1.index,'Neighbourhood'] = TorontoData.loc[Idx1.index,'Borough']

* ### Reset Index and Show Stats

In [7]:
TorontoData.reset_index(inplace=True, drop=True)
TorontoData.value_counts(['PostalCode','Borough'])

PostalCode  Borough         
M1B         Scarborough         1
M5R         Central Toronto     1
M6G         Downtown Toronto    1
M6E         York                1
M6C         York                1
                               ..
M3L         North York          1
M3K         North York          1
M3J         North York          1
M3H         North York          1
M9W         Etobicoke           1
Length: 103, dtype: int64

# Toronto Data Shape

In [8]:
TorontoData.shape

(103, 3)

# Read Toronto Postal Codes (geocoder doesn't work...)

In [9]:
# File saved locally
PostalCodes = 'C:/Users/Marco/Desktop/Geospatial_Coordinates.csv'

# Read the csv file and rename Postal Code column
GeoData = pd.read_csv(PostalCodes)
GeoData = GeoData.rename(columns={'Postal Code': 'PostalCode'})

# Create Toronto Data extended by joining the two dataframes
TorontoDataExtended = TorontoData.join(GeoData.set_index('PostalCode'), on='PostalCode')

# Show the Extended Version
TorontoDataExtended

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Create subset of Boroughs containing "Toronto" (Toronto downtown)

In [10]:
TorontoDataExtendedSubset = TorontoDataExtended.loc[TorontoDataExtended['Borough'].str.contains('Toronto')]

TorontoDataExtendedSubset.reset_index(inplace=True, drop=True)

# Search for Toronto Coordinates and create map featuring the boroughs

In [11]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
TOlat = location.latitude
TOlon = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(TOlat, TOlon))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [12]:
map_toronto = folium.Map(location=[TOlat, TOlon], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoDataExtendedSubset['Latitude'], TorontoDataExtendedSubset['Longitude'], TorontoDataExtendedSubset['Borough'], TorontoDataExtendedSubset['Neighbourhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Four-square Credentials

In [13]:
CLIENT_ID = 'NWI1F0JYOT2SILTYBSQOE2NMN0MT2A55Y01LVC4GLLGJMT3S' # your Foursquare ID
CLIENT_SECRET = 'O1CTPW5QZH4E2NCLFJCX5AXQRHVAYU4D0IQZ2U0ZVICCRSE4' # your Foursquare Secret
ACCESS_TOKEN = 'J4W3C1A2YI3XZ2JCYUPWOXYWIWZX432CU5V1TJV4CTRMO0Z4' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

# Function to search for Nearby Venues given neighbourhoods and pertaining neighbourhoods coordinates

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Apply previous function: search for Toronto Venues

In [15]:
toronto_venues = getNearbyVenues(names=TorontoDataExtendedSubset['Neighbourhood'],
                                   latitudes=TorontoDataExtendedSubset['Latitude'],
                                   longitudes=TorontoDataExtendedSubset['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

# Check for Toronto Venues DataFrame

In [16]:
print('The DataFrame has the following shape: {}\n'.format(toronto_venues.shape))
print('There are {} venues in the analyzed neighbourhood dataset.'.format(toronto_venues.shape[0]))
toronto_venues

The DataFrame has the following shape: (855, 7)

There are 855 venues in the analyzed neighbourhood dataset.


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
...,...,...,...,...,...,...,...
850,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
851,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
852,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
853,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station


# Toronto Venues Stats: check how many different venues can be found in Toronto downtown

In [17]:
print('There are {} uniques categories.\n'.format(len(toronto_venues['Venue Category'].unique())))

toronto_venues.groupby('Neighbourhood').count()

There are 190 uniques categories.



,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,30,30,30,30,30,30
Christie,15,15,15,15,15,15
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


# Analyze Each Venue and create one-hot encoded dataframe version to apply kmeans clustering

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
851,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
852,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
853,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Calculate the average distribution of venues per neighbourohood

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Farmers Market,Cocktail Bar,Coffee Shop,Greek Restaurant,Bakery,Cheese Shop,Fountain,French Restaurant,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Nightclub,Grocery Store,Convenience Store,Climbing Gym,Bar,Stadium,Bakery
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Spa,Auto Workshop,Comic Shop,Park,Recording Studio,Restaurant,Skate Park,Burrito Place,Farmers Market
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Harbor / Marina,Airport,Rental Car Location,Coffee Shop,Sculpture Garden,Boat or Ferry,Bar,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Discount Store,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Sandwich Place,Yoga Studio,Sushi Restaurant
5,Christie,Grocery Store,Café,Park,Candy Store,Coffee Shop,Restaurant,Italian Restaurant,Nightclub,Baby Store,Miscellaneous Shop
6,Church and Wellesley,Sushi Restaurant,Adult Boutique,Dance Studio,Bubble Tea Shop,Burger Joint,Café,Salon / Barbershop,Restaurant,Coffee Shop,Ramen Restaurant
7,"Commerce Court, Victoria Hotel",Hotel,Café,Coffee Shop,Restaurant,Gastropub,Steakhouse,Beer Bar,Tailor Shop,Museum,Japanese Restaurant
8,Davisville,Dessert Shop,Sushi Restaurant,Gym,Pizza Place,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Greek Restaurant,Pharmacy
9,Davisville North,Gym,Sandwich Place,Pizza Place,Breakfast Spot,Park,Hotel,Food & Drink Shop,Department Store,Middle Eastern Restaurant,Miscellaneous Shop


# Calculate kmeans: use the grouped-per-neighbourhood venues to create clusters

### Find 8 clusters of similar neighbourhoods per venues distribution

In [22]:
# set number of clusters
kclusters = 8

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# Add Cluster Number to sorted Neighbourhoods DataFame and Merge this DataFrame with Postal&Geographical DataFrame

## The final DataFrame will feature the Postal Code + Geo Info + Cluster No + First Ten common Venues

In [23]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = TorontoDataExtendedSubset

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Theater,Yoga Studio,Chocolate Shop,Farmers Market,Event Space,Café
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Café,Beer Bar,Fried Chicken Joint,Mexican Restaurant,Smoothie Shop,Burrito Place,Sandwich Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Café,Coffee Shop,Clothing Store,Theater,Mexican Restaurant,Tea Room,Burger Joint,Fast Food Restaurant,Sandwich Place,Movie Theater
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Café,Gastropub,Coffee Shop,Cocktail Bar,Farmers Market,Hotel,BBQ Joint,Park,Diner,Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,6,Neighborhood,Health Food Store,Pub,Trail,Adult Boutique,Martial Arts School,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


# Show Clusters of similar Neighbourhoods per Venues distribution on Map

In [24]:
# create map
map_clusters = folium.Map(location=[TOlat, TOlon], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters